In [11]:
import sys
import numpy as np
from math import *
import time
from PySide2.QtCore import *
from PySide2.QtGui import *
from PySide2.QtWidgets import *
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from Prog5_ui import Ui_MainWindow
from Prog3_img import Ui_Form
        
if not QApplication.instance():
    app = QApplication(sys.argv)
else:
    app = QApplication.instance()

Form = QMainWindow()
ui = Ui_MainWindow()
ui.setupUi(Form)
Form.show()

def make_graph():
    global x_0, y_0, d, N
    graph = {}
    f = open('graph', 'r')
    for line in f:
        line = line.strip().split(': ')
        graph[int(line[0])] = eval(line[1])
    x_0, y_0 = -2, 2
    d = 0.005
    N = int(4 / d)
    return graph
    
def invert_graph(graph):
    i_graph = dict.fromkeys(graph.keys(), 0)
    for k, v in graph.items():
        for vertice in v:
            if i_graph[vertice]:
                i_graph[vertice].add(int(k))
            else:
                i_graph[vertice] = {int(k)}
    for k, v in i_graph.items():
        if v == 0:
            i_graph[k] = set()
    return i_graph

def dfs1(v, comp):
    visited[v] = True
    end = True
    for e in graph[v]:
        if not visited[e]:
            end = False
            dfs1(e, comp)
    comp.append(v)
    
def dfs2(v, comp):
    visited[v] = True
    end = True
    for e in i_graph[v]:
        if not visited[e]:
            end = False
            dfs2(e, comp)
    comp.append(v)

def kosaraju(graph):
    global visited
    global i_graph
    i_graph = invert_graph(graph)
    n = N * N
    
    visited = dict.fromkeys(graph.keys(), False)
    coms = []
    
    for i in range(1, n + 1):
        if (i in graph.keys() and (not visited[i])):
            comp = []
            dfs1(i, comp)
            coms.append(comp)

    visited = dict.fromkeys(graph.keys(), False)
    c = []

    for comp in coms[::-1]:
        for v in comp[::-1]:
            if not visited[v]:
                co = []
                root = True
                dfs2(v, co)
                c.append(co)                    
    return c

def scale(x, y):
    return (x + 2) * width // 4 , height - (y + 2) * height // 4

def fill_square(number, d, color):
    y, x = divmod(number - 1, N)
    x, y = x_0 + x * d, y_0 - y * d
    draw.rectangle((*scale(x, y), *scale(x + d, y - d)), fill=color)
    
def draw_grid():
    x, y = 0, 0
    if d > 0.01:
        while x < width:
            x += width / N
            draw.line((x, 0, x, height), fill=ImageColor.getrgb("blue"))
        while y < height:
            y += height / N
            draw.line((0, x, width, x), fill=ImageColor.getrgb("blue"))
            
def draw_axis():
    zero_x = width / 2
    zero_y = height / 2

    draw.line((0, zero_x + 1, height, zero_x + 1))
    draw.line((zero_y + 1, 0, zero_y + 1, width))

def draw_scc(scc, attr_reg, grid=False, axis=False, save=True):
    global height, width, draw, cntr, cntr_edges
    height, width = 800, 800
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    draw.rectangle((0, 0, width, height), fill='white')
  
    for c in attr_reg:
        fill_square(c, d, 'pink') 
    for c in scc:
        fill_square(c, d, 'purple')     

    if axis:
        draw_axis()

    if grid:
        draw_grid()

    if save:
        im_name = 'image5_' + str(N * N) + '.png'
        image.save(im_name, "PNG")
        
def dfs3(v, attr_reg):
    visited[v] = True
    for e in i_graph[v]:
        if not visited[e]:
            dfs3(e, attr_reg)
    attr_reg.append(v)
    
def get_attr(graph, scc):
    global visited
    visited = dict.fromkeys(graph.keys(), False)
    attr_reg = []
    
    for v in scc:
        visited[v] = True

    for v in scc:
        dfs3(v, attr_reg)
        attr_reg.remove(v)

    return attr_reg

def func():
    global graph, scc, was
    start = time.time()
    graph = make_graph()
    scc = kosaraju(graph)
    short_scc = []
    for c in scc:
        if len(c) > 1:
            short_scc += c
    attr_reg = get_attr(graph, short_scc)
    draw_scc(short_scc, attr_reg)
    
#     text = ''
#     for k, v in graph.items():
#         text += f'{k}: {v}\n'
    ui.textEdit.setText(str(short_scc).strip('[]'))
#     text_2 = ''
#     for c in scc:
#         text_2 += f'{c}\n'
    ui.textEdit_2.setText(str(attr_reg).strip('[]'))
    ui.label_7.setText(f'{round(time.time() - start, 3)} c')
    
    show_image()
    
def show_image():
    global Form2
    Form2 = QWidget()
    ui2 = Ui_Form()
    ui2.setupUi(Form2)
    Form2.show()
    
    scene = QGraphicsScene()
    im_name = 'image5_' + str(N * N) + '.png'
    scene.addPixmap(QPixmap(im_name))
    ui2.graphicsView.setScene(scene)

ui.pushButton.clicked.connect(func)

sys.exit(app.exec_())


SystemExit: 0

c:\users\ekaterina\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
l = [32, 5, 4, 2]
str(l).strip('[]')

'32, 5, 4, 2'